In [2]:
# Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
uber_driver = pd.read_csv('uber_driver.csv')
uber_driver.columns

Index(['Mean_City_Hourly_Earnings', 'Month_of_Year', 'Year',
       'Temperature_Fahrenheit', 'Inches_of_Rainfall',
       'Minutes_Worked_This_Session', 'Male', 'Day_of_Week', 'Person_ID',
       'Earnings_This_Session', 'City'],
      dtype='object')

In [ ]:
# Calculating variables
uber_driver['HoursOnShift'] = uber_driver.Minutes_Worked_This_Session / 60
uber_driver['HourlyFares'] = uber_driver.Earnings_This_Session / uber_driver.HoursOnShift

In [4]:
uber_driver.Male.value_counts()

Male
1.0    844070
0.0    134502
Name: count, dtype: int64

In [2]:
import pandas as pd
import statsmodels.api as sm
# Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from linearmodels.iv import IV2SLS

# Load the data
uber_driver = pd.read_csv('uber_driver.csv')

# Define the variables based on the description in the paper and the dataset
uber_driver['HoursOnShift'] = uber_driver.Minutes_Worked_This_Session / 60
uber_driver['HourlyFares'] = uber_driver.Earnings_This_Session / uber_driver.HoursOnShift
uber_driver['logHoursOnShift'] = np.log(uber_driver['HoursOnShift'])
uber_driver['logHourlyFares'] = np.log(uber_driver['HourlyFares'])
uber_driver['logAvgHourlyFares'] = np.log(uber_driver['Mean_City_Hourly_Earnings'])

# Add quadratic term for temperature
uber_driver['temperature_squared'] = uber_driver['Temperature_Fahrenheit'] ** 2

# Define the independent variables and controls
X = uber_driver[['logHourlyFares', 'Inches_of_Rainfall', 'Temperature_Fahrenheit', 'temperature_squared']]
Z = uber_driver[['logAvgHourlyFares', 'Inches_of_Rainfall', 'Temperature_Fahrenheit', 'temperature_squared']]

# Adding fixed effects for partner, month, and day of the week
uber_driver = pd.get_dummies(uber_driver, columns=['Person_ID', 'Month_of_Year', 'Day_of_Week'], drop_first=True)

# First stage regression: log(hourly_fares) ~ log(avg_hourly_fares)
first_stage = sm.OLS(uber_driver['logHourlyFares'], sm.add_constant(Z)).fit()
uber_driver['fitted_log_hourly_fares'] = first_stage.fittedvalues

# Second stage regression: log(hours_on_shift) ~ fitted_log_hourly_fares + controls
second_stage = IV2SLS(
    uber_driver['log_hours_on_shift'],
    sm.add_constant(X),
    instrument=uber_driver['fitted_log_hourly_fares']
).fit()

# Print the regression results
print(second_stage.summary)

/Users/arnavgarg/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


: 